In [428]:
import pandas as pd
import os
from datetime import datetime
from json import loads, dumps

In [429]:
""" Only run it once when first download the data
#convert ods data into csv 
basefolder = "data/ridership-raw-data"

for folders in os.listdir(basefolder):
    for file in os.listdir(basefolder + "/" + folders):
        
        ods_file = os.path.join(basefolder,folders, file)
        # Read the ODS file
        sheets = pd.read_excel(ods_file, engine='odf', sheet_name=None)

        # Convert each sheet to CSV
        for sheet_name, df in sheets.items():
            csv_filename = f'data/csv-data/{folders}/{file[:-8]}-{sheet_name}.csv'
            df.to_csv(csv_filename, index=False)
            print(f'{sheet_name} converted to {csv_filename}')
"""

' Only run it once when first download the data\n#convert ods data into csv \nbasefolder = "data/ridership-raw-data"\n\nfor folders in os.listdir(basefolder):\n    for file in os.listdir(basefolder + "/" + folders):\n        \n        ods_file = os.path.join(basefolder,folders, file)\n        # Read the ODS file\n        sheets = pd.read_excel(ods_file, engine=\'odf\', sheet_name=None)\n\n        # Convert each sheet to CSV\n        for sheet_name, df in sheets.items():\n            csv_filename = f\'data/csv-data/{folders}/{file[:-8]}-{sheet_name}.csv\'\n            df.to_csv(csv_filename, index=False)\n            print(f\'{sheet_name} converted to {csv_filename}\')\n'

In [430]:
dow = pd.read_csv("data/day-of-week-2023.csv")

# merge all tables together
for folder in os.listdir("data/csv-data"):
    exitList = [] # list of files that contain exit station count
    enterList = [] # list of files that contain enter station count
    if folder == "2023":

        for file in os.listdir(os.path.join("data/csv-data", folder)):
            if "出站" in file:
                filepath = os.path.join("data/csv-data", folder, file)
                month = pd.read_csv(filepath)
                exitList.append(month)
            elif "進站" in file:
                filepath = os.path.join("data/csv-data", folder, file)
                month = pd.read_csv(filepath)
                enterList.append(month)

        exit_result = pd.concat(exitList, ignore_index= True, sort = False)
        enter_result = pd.concat(enterList, ignore_index= True, sort = False)
            


In [431]:
frames = [exit_result, enter_result]
sheetName = ["exit", "enter"]

In [432]:
#add station coordinates to the data
conversion = pd.read_csv("data/conversion-chart.csv")
a = 0
transposedList  = []
for frame in frames:

    frame = frame.drop(["十四張","秀朗橋","景平","中和","橋和","中原","板新","Y板橋","新埔民生","幸福","新北產業園區"], axis =1)
    # Merge Stations, some stations are counted separately from the rest of the system (i.e stations on Yellow lines) (i.e. BL 板橋 and Y 板橋) should be counted as 1 station. 
    transposed_result = frame.T.reset_index() #.to_csv("processed/day-of-week.csv")

    # clean up the transposed dataframe 
    columns = transposed_result.iloc[0]
    transposed_result.columns = columns
    transposed_result = transposed_result.drop(transposed_result.index[0])
    
    transposed_cleaned = pd.merge(transposed_result, conversion, left_on = "　　　　車站日期", right_on = "Station")
    #some station have more than one row of data, so sum them together
    columns = transposed_cleaned.columns.tolist() #get field names
    transposed_cleaned = pd.pivot_table(transposed_cleaned, index = ["Station Code", "x", "y"], values = columns[1:-4], aggfunc= "sum", sort = False).reset_index()
    
     #Sort values by day of week
    station_dow = transposed_cleaned[["Station Code", "x", "y"]]
    station_dow["x"] = station_dow["x"].astype('float')

    monday = []
    tuesday = []
    wednesday = []
    thursday = []
    friday = []
    saturday = []
    sunday = []

    for date in transposed_cleaned.columns.tolist()[3:]:

        date_object = datetime.strptime(date, '%Y-%m-%d').date()
        match date_object.strftime('%A'):
            case "Monday":
                monday.append(date)
            case "Tuesday":
                tuesday.append(date)
            case "Wednesday":
                wednesday.append(date)
            case "Thursday":
                thursday.append(date)
            case "Friday":
                friday.append(date)
            case "Saturday":
                saturday.append(date)
            case "Sunday":
                sunday.append(date)

    #get average ridership per day
    station_dow[f'Monday-{sheetName[a]}'] = transposed_cleaned[monday].sum(axis=1)/len(monday)
    station_dow[f'Tuesday-{sheetName[a]}'] = transposed_cleaned[tuesday].sum(axis=1)/len(tuesday)
    station_dow[f'Wednesday-{sheetName[a]}'] = transposed_cleaned[wednesday].sum(axis=1)/len(wednesday)
    station_dow[f'Thursday-{sheetName[a]}'] = transposed_cleaned[thursday].sum(axis=1)/len(thursday)
    station_dow[f'Friday-{sheetName[a]}'] = transposed_cleaned[friday].sum(axis=1)/len(friday)
    station_dow[f'Saturday-{sheetName[a]}'] = transposed_cleaned[saturday].sum(axis=1)/len(saturday)
    station_dow[f'Sunday-{sheetName[a]}'] = transposed_cleaned[sunday].sum(axis=1)/len(sunday)
    a += 1
    
    transposedList.append(station_dow)


C:\Users\589dl\AppData\Local\Temp\ipykernel_12460\1929692860.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_dow["x"] = station_dow["x"].astype('float')
C:\Users\589dl\AppData\Local\Temp\ipykernel_12460\1929692860.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_dow[f'Monday-{sheetName[a]}'] = transposed_cleaned[monday].sum(axis=1)/len(monday)
C:\Users\589dl\AppData\Local\Temp\ipykernel_12460\1929692860.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [433]:
transposedList[1] = transposedList[1].drop(["x", "y"], axis = 1)
enter_exit_count = pd.merge(transposedList[0], transposedList[1], on = "Station Code")

In [434]:
enter_exit_count

,Station Code,x,y,Monday-exit,Tuesday-exit,Wednesday-exit,Thursday-exit,Friday-exit,Saturday-exit,Sunday-exit,Monday-enter,Tuesday-enter,Wednesday-enter,Thursday-enter,Friday-enter,Saturday-enter,Sunday-enter
0,BR13 松山機場,121.552010,25.062908,4761.0,4604.0,4777.269231,4719.25,5093.980769,3714.711538,3509.264151,4724.942308,4698.192308,4813.384615,4669.903846,5186.634615,3907.057692,3727.150943
1,BR12 中山國中,121.544215,25.060850,12681.019231,13442.115385,13863.846154,13618.134615,13988.634615,9274.480769,7282.962264,12515.480769,13270.192308,13646.903846,13447.134615,13782.980769,9348.288462,7282.716981
2,BR11 G16 南京復興,121.544303,25.052044,39858.769231,42122.173077,43466.096154,42949.769231,44857.153846,27277.192308,19609.603774,37789.326923,39758.384615,40890.673077,40344.653846,41373.596154,25328.25,19050.471698
3,BR10 BL15 忠孝復興,121.545026,25.041749,38301.807692,39565.25,40464.75,40997.692308,46843.615385,45015.673077,34684.358491,39068.153846,40267.25,41177.923077,41672.692308,46948.403846,45311.615385,35973.283019
4,BR09 R05 大安,121.542370,25.033311,22828.538462,24140.038462,24848.076923,24655.423077,25559.692308,17154.038462,12505.509434,22621.057692,23954.269231,24799.903846,24526.75,25301.942308,16345.807692,11952.169811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,O52 徐匯中學,121.479945,25.080485,11933.211538,12296.269231,12529.480769,12379.038462,12936.115385,10391.615385,8435.320755,11197.596154,11589.019231,11794.307692,11721.442308,12370.538462,10135.538462,8048.226415
104,O51 三和國中,121.486795,25.076460,11490.25,11764.038462,12073.346154,11944.75,12184.5,9473.192308,7737.037736,11745.173077,12058.307692,12368.423077,12287.461538,12697.865385,9844.903846,7825.433962
105,O50 三重國小,121.497010,25.070275,11176.326923,11416.442308,11703.519231,11495.615385,11924.75,9737.826923,8032.150943,10826.192308,11130.769231,11401.0,11268.461538,11780.596154,9714.134615,7857.641509
106,O21 迴龍,121.411757,25.022107,10210.25,10478.615385,10698.673077,10556.711538,10806.519231,7983.5,6594.622642,10305.423077,10671.711538,10929.615385,10813.576923,11260.461538,7923.5,6333.698113


In [435]:
def determineLine (code):
    if "BR" in code:
        routes="文湖線"
    elif "BL" in code:
        routes="板南線"
    elif "O" in code:
        routes="中和新蘆線"
    elif "G" in code:
        routes="松山新店線"
    elif "Y" in code: 
        routes="環狀線"
    elif "R" in code:
        routes="淡水信義線"
    return routes

In [436]:
stationList = []
for station in enter_exit_count['Station Code'].values.tolist():
    name = station.split(" ")[-1]
    stationList.append(name)

# add a column where there is only station names

enter_exit_count.insert(0,"Station", stationList)
enter_exit_count = enter_exit_count.sort_values("Station").reset_index(drop=True)
enter_exit_count

,Station,Station Code,x,y,Monday-exit,Tuesday-exit,Wednesday-exit,Thursday-exit,Friday-exit,Saturday-exit,Sunday-exit,Monday-enter,Tuesday-enter,Wednesday-enter,Thursday-enter,Friday-enter,Saturday-enter,Sunday-enter
0,七張,G03 七張,121.542985,24.975450,14827.096154,15206.653846,15455.615385,15241.807692,15866.192308,12471.153846,10653.320755,14014.384615,14443.211538,14783.480769,14574.096154,15491.057692,12486.769231,10255.867925
1,三和國中,O51 三和國中,121.486795,25.076460,11490.25,11764.038462,12073.346154,11944.75,12184.5,9473.192308,7737.037736,11745.173077,12058.307692,12368.423077,12287.461538,12697.865385,9844.903846,7825.433962
2,三民高中,O53 三民高中,121.473365,25.085425,11487.326923,11872.538462,12135.480769,12016.115385,12395.615385,9431.307692,7548.792453,11078.057692,11453.576923,11694.115385,11581.730769,12054.826923,9442.519231,7343.698113
3,三重,O15 三重,121.484220,25.055710,7214.192308,7425.0,7540.461538,7501.461538,7808.807692,7456.423077,6504.528302,7098.519231,7319.307692,7438.0,7424.692308,7784.307692,7145.615385,6192.811321
4,三重國小,O50 三重國小,121.497010,25.070275,11176.326923,11416.442308,11703.519231,11495.615385,11924.75,9737.826923,8032.150943,10826.192308,11130.769231,11401.0,11268.461538,11780.596154,9714.134615,7857.641509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,頂埔,BL01 頂埔,121.420500,24.960120,8595.115385,8842.269231,9019.615385,8908.019231,9296.019231,6477.211538,5680.566038,9011.192308,9254.961538,9441.961538,9330.730769,9702.192308,6661.519231,5634.169811
104,頂溪,O04 頂溪,121.515462,25.013858,32463.980769,33496.596154,34121.634615,33656.365385,34853.461538,28331.730769,23984.924528,30859.076923,32015.346154,32659.269231,32448.5,34017.384615,28153.673077,22785.207547
105,頭前庄,O17 Y18 頭前庄,121.461655,25.039735,2261.0,2334.980769,2367.211538,2341.442308,2394.442308,1909.884615,1622.830189,5204.903846,5346.884615,5457.25,5409.980769,5574.442308,4199.057692,3393.396226
106,麟光,BR06 麟光,121.558834,25.018495,3824.711538,3858.365385,3952.134615,3880.692308,3990.384615,3646.096154,3332.09434,3985.769231,4073.673077,4178.153846,4136.903846,4357.884615,3846.038462,3280.792453


In [437]:
stationCodes = []
routesList = []
i = 0

for station in enter_exit_count['Station Code'].values.tolist():
    items = station.split(" ") 
    stationName = items[-1]
    items.remove(items[-1]) # remove station name
    print("All Items: ", items)
    if len(items) == 1: #when only the station only service one transit route
        i += 1
    else: #when there is more than 1 code (a transfer station)
        a = 1
        while a < len(items): #add the 2nd, 3rd, 4th code
            print("Copying ", enter_exit_count.loc[i]['Station Code'])
            frameLength = len(enter_exit_count['Station Code'].values.tolist())
            enter_exit_count.loc[frameLength] = enter_exit_count.loc[i].copy()
            print("New Data Frame Length", frameLength)
            a+=1
        print (" ")
        i+=1
        
enter_exit_count = enter_exit_count.sort_values("Station").reset_index(drop=True)

All Items:  ['G03']
All Items:  ['O51']
All Items:  ['O53']
All Items:  ['O15']
All Items:  ['O50']
All Items:  ['R11', 'G14']
Copying  R11 G14 中山
New Data Frame Length 108
 
All Items:  ['BR12']
All Items:  ['O10']
All Items:  ['R08', 'G10']
Copying  R08 G10 中正紀念堂
New Data Frame Length 109
 
All Items:  ['O20']
All Items:  ['BL05']
All Items:  ['R04']
All Items:  ['O16']
All Items:  ['BR19']
All Items:  ['G07']
All Items:  ['BR07']
All Items:  ['BR15']
All Items:  ['R15']
All Items:  ['BR01']
All Items:  ['R22']
All Items:  ['G13']
All Items:  ['G18']
All Items:  ['BR11', 'G16']
Copying  BR11 G16 南京復興
New Data Frame Length 110
 
All Items:  ['O01']
All Items:  ['BL22']
All Items:  ['BR24', 'BL23']
Copying  BR24 BL23 南港展覽館
New Data Frame Length 111
 
All Items:  ['BR23']
All Items:  ['G09', 'O05']
Copying  G09 O05 古亭
New Data Frame Length 112
 
All Items:  ['R03']
All Items:  ['G17']
All Items:  ['O13']
All Items:  ['R10', 'BL12']
Copying  R10 BL12 台北車站
New Data Frame Length 113
 
All 

In [438]:
enter_exit_count.head(30)

,Station,Station Code,x,y,Monday-exit,Tuesday-exit,Wednesday-exit,Thursday-exit,Friday-exit,Saturday-exit,Sunday-exit,Monday-enter,Tuesday-enter,Wednesday-enter,Thursday-enter,Friday-enter,Saturday-enter,Sunday-enter
0,七張,G03 七張,121.542985,24.975450,14827.096154,15206.653846,15455.615385,15241.807692,15866.192308,12471.153846,10653.320755,14014.384615,14443.211538,14783.480769,14574.096154,15491.057692,12486.769231,10255.867925
1,三和國中,O51 三和國中,121.486795,25.076460,11490.25,11764.038462,12073.346154,11944.75,12184.5,9473.192308,7737.037736,11745.173077,12058.307692,12368.423077,12287.461538,12697.865385,9844.903846,7825.433962
2,三民高中,O53 三民高中,121.473365,25.085425,11487.326923,11872.538462,12135.480769,12016.115385,12395.615385,9431.307692,7548.792453,11078.057692,11453.576923,11694.115385,11581.730769,12054.826923,9442.519231,7343.698113
3,三重,O15 三重,121.484220,25.055710,7214.192308,7425.0,7540.461538,7501.461538,7808.807692,7456.423077,6504.528302,7098.519231,7319.307692,7438.0,7424.692308,7784.307692,7145.615385,6192.811321
4,三重國小,O50 三重國小,121.497010,25.070275,11176.326923,11416.442308,11703.519231,11495.615385,11924.75,9737.826923,8032.150943,10826.192308,11130.769231,11401.0,11268.461538,11780.596154,9714.134615,7857.641509
5,中山,R11 G14 中山,121.520364,25.052621,41025.326923,42369.057692,43419.923077,43787.653846,51070.307692,53198.076923,43521.056604,40390.5,41591.538462,42568.211538,42834.076923,49174.653846,50958.865385,42332.150943
6,中山,R11 G14 中山,121.520364,25.052621,41025.326923,42369.057692,43419.923077,43787.653846,51070.307692,53198.076923,43521.056604,40390.5,41591.538462,42568.211538,42834.076923,49174.653846,50958.865385,42332.150943
7,中山國中,BR12 中山國中,121.544215,25.060850,12681.019231,13442.115385,13863.846154,13618.134615,13988.634615,9274.480769,7282.962264,12515.480769,13270.192308,13646.903846,13447.134615,13782.980769,9348.288462,7282.716981
8,中山國小,O10 中山國小,121.526609,25.062665,15704.692308,16469.288462,16894.711538,16797.461538,17891.211538,13831.711538,10860.339623,15788.634615,16472.711538,16842.942308,16748.75,17572.75,13512.730769,10824.377358
9,中正紀念堂,R08 G10 中正紀念堂,121.518273,25.032767,23339.096154,25740.288462,25733.596154,25586.903846,26904.365385,22209.942308,19381.773585,23552.826923,26021.942308,26165.173077,26169.173077,27754.384615,22428.769231,19076.660377


In [439]:
# get the station codes and line names

stationCodes = []
routesList = []
i = 0
station_codes = enter_exit_count['Station Code'].values.tolist()
while i < len(station_codes):

    items = station_codes[i].split(" ") 

    if i > 0:
        a = 0
        if station_codes[i] == station_codes[i-1]:
            a +=1
            stationCodes.append(items[a])
            routesList.append(determineLine(items[a]))
            print(items[-1], f"({items[a]}) is a transfer station of ", determineLine(items[a]))
            
        else:
            print(items[-1], f"({items[a]}) is a station of ", determineLine(items[a]))
            stationCodes.append(items[a])
            routesList.append(determineLine(items[0]))
    else:
        print(items[-1], f"({items[0]}) is a station of ", determineLine(items[0]))
        stationCodes.append(items[0])
        routesList.append(determineLine(items[0]))

    i+=1

七張 (G03) is a station of  松山新店線
三和國中 (O51) is a station of  中和新蘆線
三民高中 (O53) is a station of  中和新蘆線
三重 (O15) is a station of  中和新蘆線
三重國小 (O50) is a station of  中和新蘆線
中山 (R11) is a station of  淡水信義線
中山 (G14) is a transfer station of  松山新店線
中山國中 (BR12) is a station of  文湖線
中山國小 (O10) is a station of  中和新蘆線
中正紀念堂 (R08) is a station of  淡水信義線
中正紀念堂 (G10) is a transfer station of  松山新店線
丹鳳 (O20) is a station of  中和新蘆線
亞東醫院 (BL05) is a station of  板南線
信義安和 (R04) is a station of  淡水信義線
先嗇宮 (O16) is a station of  中和新蘆線
內湖 (BR19) is a station of  文湖線
公館 (G07) is a station of  松山新店線
六張犁 (BR07) is a station of  文湖線
劍南路 (BR15) is a station of  文湖線
劍潭 (R15) is a station of  淡水信義線
動物園 (BR01) is a station of  文湖線
北投 (R22) is a station of  淡水信義線
北門 (G13) is a station of  松山新店線
南京三民 (G18) is a station of  松山新店線
南京復興 (BR11) is a station of  文湖線
南京復興 (G16) is a transfer station of  松山新店線
南勢角 (O01) is a station of  中和新蘆線
南港 (BL22) is a station of  板南線
南港展覽館 (BR24) is a station of  文湖線
南港展覽館 (BL23) is a tr

In [440]:
enter_exit_count.insert(1,"Code", stationCodes)
enter_exit_count.insert(2,"Route", routesList)

#enter_exit_count = enter_exit_count.sort_values("Station").reset_index(drop=True)

enter_exit_count = enter_exit_count.sort_values("Code").reset_index(drop=True)


In [441]:
enter_exit_count.head(40)

,Station,Code,Route,Station Code,x,y,Monday-exit,Tuesday-exit,Wednesday-exit,Thursday-exit,Friday-exit,Saturday-exit,Sunday-exit,Monday-enter,Tuesday-enter,Wednesday-enter,Thursday-enter,Friday-enter,Saturday-enter,Sunday-enter
0,頂埔,BL01,板南線,BL01 頂埔,121.420500,24.960120,8595.115385,8842.269231,9019.615385,8908.019231,9296.019231,6477.211538,5680.566038,9011.192308,9254.961538,9441.961538,9330.730769,9702.192308,6661.519231,5634.169811
1,永寧,BL02,板南線,BL02 永寧,121.436130,24.966820,12337.576923,12736.692308,13005.25,12930.519231,13981.25,10880.653846,8937.018868,12318.384615,12536.730769,12741.173077,12646.192308,13456.538462,10532.519231,9122.396226
2,土城,BL03,板南線,BL03 土城,121.444320,24.973130,7870.384615,7959.538462,8173.788462,7976.769231,8400.826923,6590.0,5377.09434,7918.692308,7995.711538,8233.961538,8025.769231,8486.769231,6801.480769,5409.132075
3,海山,BL04,板南線,BL04 海山,121.448730,24.985305,21357.076923,21954.769231,22405.615385,22176.442308,22831.461538,17458.019231,14357.716981,20605.134615,21265.346154,21701.942308,21557.538462,22326.711538,17499.826923,13959.90566
4,亞東醫院,BL05,板南線,BL05 亞東醫院,121.452465,24.998280,21949.480769,22837.346154,23309.403846,22779.826923,22808.173077,15465.884615,11690.90566,23683.576923,24635.173077,25210.865385,24665.557692,25168.653846,17044.346154,12472.641509
5,府中,BL06,板南線,BL06 府中,121.459276,25.008465,24462.288462,25274.519231,25541.788462,25303.826923,27498.153846,25419.519231,20510.584906,26998.903846,27959.384615,28276.788462,28023.134615,30068.384615,27610.230769,22738.773585
6,板橋,BL07,板南線,BL07 Y16 板橋,121.462305,25.013825,38470.461538,38577.384615,39296.019231,39304.692308,44231.442308,42213.942308,35314.509434,35199.692308,35029.826923,35452.307692,35297.288462,39989.653846,39285.423077,34942.283019
7,新埔,BL08,板南線,BL08 新埔,121.468055,25.023270,34782.711538,36138.788462,36872.634615,36362.038462,37371.038462,28932.692308,23710.528302,35516.192308,36980.480769,37865.788462,37456.423077,39170.326923,30432.134615,24119.188679
8,江子翠,BL09,板南線,BL09 江子翠,121.472570,25.030265,25595.442308,26442.480769,27207.480769,26629.884615,27520.480769,21878.942308,17890.641509,24509.769231,25391.5,26143.961538,25711.384615,26908.403846,21696.673077,17184.264151
9,龍山寺,BL10,板南線,BL10 龍山寺,121.500325,25.035280,25538.480769,25531.442308,25964.346154,25899.980769,27370.75,26624.615385,23586.943396,25873.307692,25960.730769,26334.557692,26333.980769,27889.115385,27396.75,24190.792453


In [442]:
with open("data-json/2023-ridership.json", 'w', encoding = 'utf-8') as file: 
    enter_exit_count.to_json(file, orient="records",force_ascii=False,)
